## Pydantic Basics

Pydantic is the most widely used data validation library for Python.

Main features:

- IDE Type Hints
- Data Validation
- JSON Serialization

In [1]:
import os
import json
from pydantic import (
    BaseModel,
    EmailStr,
    AnyUrl,
    SecretStr,
    field_validator,
    field_serializer,
    Field,
    computed_field,
)
from pydantic_settings import BaseSettings, SettingsConfigDict
from typing import List, Literal, Optional
from datetime import date, datetime

In [2]:
class UserProfile(BaseModel):
    username: str = Field(
        description="Platform user name.",
        examples=["lucas", "matthew"],
        min_length=5,
        max_length=20,
    )
    email: EmailStr = Field(description="Platform user e-mail.", frozen=True)
    id: int = Field(lt=1000)
    age: int = Field(ge=12, le=99)
    active: bool = True
    website: Optional[AnyUrl] = None
    join_date: date
    last_login: Optional[datetime] = None
    user_category: Literal["free", "premium"] = "free"
    # password: SecretStr = Field(exclude=True)

    @computed_field
    @property
    def legal_age(self) -> bool:
        if self.age >= 18:
            return True
        else:
            return False

    @field_validator("id")
    @classmethod
    def validate_id(cls, value):
        if value <= 0:
            raise ValueError(f"User ID must be > 0 ! Actual value: {value}")
        return value

    @field_serializer("username", when_used="json")
    @classmethod
    def serialize_username(cls, username):
        return username.upper()

In [3]:
user_data = {
    "username": "johnsmith",
    "email": "john@mail.com",
    "id": 123,
    "age": 30,
    "active": False,
    "website": "https://mywebsite.com",
    "join_date": date(2025, 7, 26),
    "last_login": datetime(2025, 7, 26, 10, 0),
    "user_category": "free",
    # "password": "pass123"
}

In [4]:
user = UserProfile(**user_data)

In [5]:
UserProfile.model_validate(user_data, strict=True)

UserProfile(username='johnsmith', email='john@mail.com', id=123, age=30, active=False, website=AnyUrl('https://mywebsite.com/'), join_date=datetime.date(2025, 7, 26), last_login=datetime.datetime(2025, 7, 26, 10, 0), user_category='free', legal_age=True)

In [6]:
print(user)
print("-----")
print(user.username)
print(user.email)
print(user.id)
print(user.age)
print(user.active)
print(user.website)
print(user.join_date)
print(user.last_login)
print(user.user_category)
# print(user.password)

username='johnsmith' email='john@mail.com' id=123 age=30 active=False website=AnyUrl('https://mywebsite.com/') join_date=datetime.date(2025, 7, 26) last_login=datetime.datetime(2025, 7, 26, 10, 0) user_category='free' legal_age=True
-----
johnsmith
john@mail.com
123
30
False
https://mywebsite.com/
2025-07-26
2025-07-26 10:00:00
free


In [7]:
# convert pydantic model to JSON
user_json_str = user.model_dump_json()

# convert pydantic model to python dictionary object
user_json_dict = user.model_dump()

In [8]:
print(f"JSON String:\n{user_json_str}")
print(f"\nJSON Dictionary:\n{user_json_dict}")

JSON String:
{"username":"JOHNSMITH","email":"john@mail.com","id":123,"age":30,"active":false,"website":"https://mywebsite.com/","join_date":"2025-07-26","last_login":"2025-07-26T10:00:00","user_category":"free","legal_age":true}

JSON Dictionary:
{'username': 'johnsmith', 'email': 'john@mail.com', 'id': 123, 'age': 30, 'active': False, 'website': AnyUrl('https://mywebsite.com/'), 'join_date': datetime.date(2025, 7, 26), 'last_login': datetime.datetime(2025, 7, 26, 10, 0), 'user_category': 'free', 'legal_age': True}


In [9]:
# convert JSON string object to pydantic model
new_user = UserProfile.model_validate_json(user_json_str)
new_user

UserProfile(username='JOHNSMITH', email='john@mail.com', id=123, age=30, active=False, website=AnyUrl('https://mywebsite.com/'), join_date=datetime.date(2025, 7, 26), last_login=datetime.datetime(2025, 7, 26, 10, 0), user_category='free', legal_age=True)

In [10]:
# convert python dictionary object to pydantic model
new_user = UserProfile.model_validate(user_json_dict)
new_user

UserProfile(username='johnsmith', email='john@mail.com', id=123, age=30, active=False, website=AnyUrl('https://mywebsite.com/'), join_date=datetime.date(2025, 7, 26), last_login=datetime.datetime(2025, 7, 26, 10, 0), user_category='free', legal_age=True)

In [11]:
# generate JSON schema as a dictionary
schema_dict = new_user.model_json_schema()

# optionally, serialize to a JSON string
schema_json = json.dumps(schema_dict, indent=2)

print(schema_json)

{
  "properties": {
    "username": {
      "description": "Platform user name.",
      "examples": [
        "lucas",
        "matthew"
      ],
      "maxLength": 20,
      "minLength": 5,
      "title": "Username",
      "type": "string"
    },
    "email": {
      "description": "Platform user e-mail.",
      "format": "email",
      "title": "Email",
      "type": "string"
    },
    "id": {
      "exclusiveMaximum": 1000,
      "title": "Id",
      "type": "integer"
    },
    "age": {
      "maximum": 99,
      "minimum": 12,
      "title": "Age",
      "type": "integer"
    },
    "active": {
      "default": true,
      "title": "Active",
      "type": "boolean"
    },
    "website": {
      "anyOf": [
        {
          "format": "uri",
          "minLength": 1,
          "type": "string"
        },
        {
          "type": "null"
        }
      ],
      "default": null,
      "title": "Website"
    },
    "join_date": {
      "format": "date",
      "title": "Join Date"

## Pydantic Settings Management

In [12]:
os.environ["AWS_ACCESS_KEY_ID"] = (
    "my-access-key-id-value (readed from ENVIRONMENT variable)"
)
os.environ["AWS_SECRET_ACCESS_KEY"] = (
    "my-secret-access-key-value (readed from ENVIRONMENT variable)"
)
os.environ["AWS_SESSION_TOKEN"] = (
    "my-session-token-value (readed from ENVIRONMENT variable)"
)
# os.environ["AWS_DEFAULT_REGION"] = "us-east-1 (readed from ENVIRONMENT variable)"

In [13]:
class ConfigSetup(BaseSettings):
    model_config = SettingsConfigDict(
        env_file=".env", env_file_encoding="utf-8", extra="ignore"
    )

    var_1: str = Field(
        default="default_value", validation_alias="aws_access_key_id"
    )
    var_2: str = Field(default="default_value", alias="aws_session_token")
    var_3: str = Field(default="default_value")
    var_4: str = Field(default="default_value", alias="aws_default_region")

In [14]:
my_config = ConfigSetup()
print(my_config.model_dump())

{'var_1': 'my-access-key-id-value (readed from ENVIRONMENT variable)', 'var_2': 'my-session-token-value (readed from ENVIRONMENT variable)', 'var_3': 'default_value', 'var_4': 'us-east-1 (readed from .env file)'}
